In [1]:
# !pip install git+https://github.com/UpalabdhaD/DNAflexpy.git

  Cloning https://github.com/UpalabdhaD/DNAflexpy.git to /tmp/pip-req-build-owu6aldy
  Running command git clone --filter=blob:none --quiet https://github.com/UpalabdhaD/DNAflexpy.git /tmp/pip-req-build-owu6aldy
  Resolved https://github.com/UpalabdhaD/DNAflexpy.git to commit c1ab6bd2ac1f9d6dcebbd2d4c6b6c8c621ea7301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for DNAflexpy: filename=dnaflexpy-0.1-py3-none-any.whl size=10706 sha256=2ee6661bc4a44b045684b9525fe73fcf7e49b87057f37f1870f5ddde74bdbfed
  Stored in directory: /tmp/pip-ephem-wheel-cache-o7l15vmy/wheels/34/ea/ec/638c5f252cfec21fa11a959abbb319e2d9ef3738cc6ff858f2
Successfully built DNAflexpy


In [1]:
# -*- coding: utf-8 -*-
"""TF_FAMILY_SPECIFIC_XGB.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1aPsqH0Qtry3SbPmwBt_bzwvSlFs2ufCX
"""

# prompt: mount drive

# from google.colab import drive
# drive.mount('/content/drive')

# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py --prerelease if-necessary

# !pip install Bio
# !pip install git+https://github.com/UpalabdhaD/DNAflexpy.git





# Commented out IPython magic to ensure Python compatibility.
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
import json # <<< Added for JSON output
import traceback # For printing full tracebacks on error

# --- GPU ACCELERATION SETUP ---
# Attempt to automatically patch scikit-learn functions with cuML equivalents
# This might accelerate GridSearchCV, StratifiedKFold, metrics etc. if compatible versions exist.
try:
    %load_ext cuml.accel
    # If running as a script, explicit patching might be needed if desired,
    # but XGBoost GPU usage is handled separately.
    print("Attempted to load cuml.accel extension (effective in IPython/Jupyter).")
except Exception as e:
    print(f"Could not load cuml.accel: {e}. Proceeding without automatic patching.")
    print("XGBoost GPU acceleration will still be attempted if configured.")
# -----------------------------

# Scikit-learn imports (might be patched by cuml.accel)
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, matthews_corrcoef
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

# Import XGBoost separately
from xgboost import XGBClassifier

# Import tqdm for progress bars
from tqdm import tqdm

# Import cupy for potential explicit data conversion if needed
try:
    import cupy as cp
    print("CuPy imported successfully.")
except ImportError:
    print("CuPy not found. Explicit GPU array conversion will not be possible.")
    cp = None # Define cp as None if not available

# Assuming DNAflexpy is available in the environment
try:
    from DNAflexpy.core import DNAflexpyMP as DNAflexpy
except ImportError:
    print("Warning: DNAflexpy not found. Feature extraction requiring it will fail.")
    # Define a placeholder or handle the error as appropriate
    DNAflexpy = None

# Define the path to the directory containing the FASTA files
# Adjust this path as necessary
# data_dir = "../../RESULTS/INVIVO_ENCODE"
# Use the provided path for Colab example
data_dir='/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/XGB_ENCODE_RESULTS' # <<< Updated path

if not os.path.isdir(data_dir):
    print(f"Warning: Data directory not found at {data_dir}")

# Define the JSON output file name <<< Added
JSON_BASE_FILENAME = "nested_cv_fold_results.jsonl"
JSON_FULL_PATH = os.path.join(data_dir, JSON_BASE_FILENAME)
TARGET_FOLDS = 10 # <<< Define the expected number of folds for completion

# --- FUNCTIONS ---

# --- START: Function to load completed folds --- <<< NEW FUNCTION
def load_completed_folds(json_filename):
    """Reads the JSON Lines log file and counts completed folds per combination."""
    completed_counts = {}
    if not os.path.exists(json_filename):
        print(f"Log file {json_filename} not found. Starting fresh.")
        return completed_counts

    print(f"Reading existing results from {json_filename}...")
    with open(json_filename, 'r') as f_json:
        for line_num, line in enumerate(f_json):
            line = line.strip()
            if not line:
                continue # Skip empty lines
            try:
                record = json.loads(line)
                # Ensure necessary keys exist (handle potential malformed lines)
                if all(k in record for k in ['tf', 'cell_line', 'features', 'fold']):
                    # Use the same feature string format as stored in JSON
                    features_str = record['features']
                    key = (record['tf'], record['cell_line'], features_str)
                    completed_counts[key] = completed_counts.get(key, 0) + 1
                else:
                    print(f"  Warning: Skipping malformed record on line {line_num+1} (missing keys).")
            except json.JSONDecodeError:
                print(f"  Warning: Skipping invalid JSON on line {line_num+1}: {line[:100]}...") # Print start of bad line
            except Exception as e:
                 print(f"  Warning: Unexpected error processing line {line_num+1}: {e}")

    print(f"Found {len(completed_counts)} combinations with existing results.")
    return completed_counts
    
# --- END: Function to load completed folds ---

def manual_one_hot_encoding(seq, alphabet=['A', 'C', 'G', 'T']):
    """Manually one-hot encodes a DNA sequence."""
    mapping = {nt: i for i, nt in enumerate(alphabet)}
    one_hot = np.zeros((len(seq), len(alphabet)), dtype=np.float32) # Use float32 for GPU compatibility
    for i, nt in enumerate(seq):
        nt_upper = nt.upper() # Ensure case-insensitivity
        if nt_upper in mapping:
            one_hot[i, mapping[nt_upper]] = 1.0
    return one_hot.flatten()

def one_hot_encode_fasta(fa_filepath, label=1, alphabet=['A', 'C', 'G', 'T']):
    """
    One-hot encodes DNA sequences from a FASTA file.
    Returns a dictionary {seq_id: {'id': seq_id, 'feat': np.array, 'label': label}}.
    """
    encoded_data = {}
    if not os.path.exists(fa_filepath):
        print(f"Warning: FASTA file not found: {fa_filepath}")
        return encoded_data

    try:
        with open(fa_filepath, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                seq_id = record.id
                seq = str(record.seq)

                # One-hot encode the sequence
                one_hot_features = manual_one_hot_encoding(seq, alphabet)

                # Store in the dictionary
                encoded_data[seq_id] = {'id': seq_id,
                                        'feat': one_hot_features,
                                        'label': label}
    except Exception as e:
        print(f"Error reading or parsing FASTA file {fa_filepath}: {e}")
        return {} # Return empty dict on error
    return encoded_data

def extract_and_combine_features(tf, cell_line, features, alphabet=['A', 'C', 'G', 'T']):
    """
    Extracts features (starting with one-hot encoding) and combines them.
    Handles duplicate sequence IDs in flexibility data by keeping the first occurrence.
    Returns NumPy arrays X, y.
    """
    print(f"Processing {tf} - {cell_line}...")
    # Construct full paths within the function
    sequences_dir = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/INVIVO_ENCODE/SEQUENCES"
    bound_file = os.path.join(sequences_dir, f"{tf}_{cell_line}_BOUND_20.fa")
    unbound_file = os.path.join(sequences_dir, f"{tf}_{cell_line}_UNBOUND_20.fa")

    # Check if SEQUENCES directory exists
    if not os.path.isdir(sequences_dir):
         print(f"Error: SEQUENCES directory not found at {sequences_dir}")
         return np.array([]), np.array([])

    # One-hot encode the sequences
    bound_file_ohe = one_hot_encode_fasta(bound_file, label=1, alphabet=alphabet)
    unbound_file_ohe = one_hot_encode_fasta(unbound_file, label=0, alphabet=alphabet)

    if not bound_file_ohe or not unbound_file_ohe:
        print(f"Warning: Missing or empty bound/unbound OHE data for {tf} - {cell_line}. Skipping feature extraction.")
        return np.array([]), np.array([]) # Return empty arrays

    # Sample equal number of unbound data
    bound_size = len(bound_file_ohe)
    # Ensure unbound_file_ohe is not empty before checking length
    if unbound_file_ohe and len(unbound_file_ohe) < bound_size:
         print(f"Warning: Fewer unbound samples ({len(unbound_file_ohe)}) than bound samples ({bound_size}). Using all unbound.")
         # No resampling needed if unbound is smaller
    elif unbound_file_ohe: # Only resample if unbound_file_ohe is not empty and >= bound_size
         unbound_items = list(unbound_file_ohe.items())
         unbound_file_ohe_items = resample(unbound_items,
                                        n_samples=bound_size,
                                        replace=False, # No replacement if unbound >= bound
                                        random_state=42)
         unbound_file_ohe = dict(unbound_file_ohe_items)
    # else: unbound_file_ohe is empty, handled by the initial check


    combined_data = {**bound_file_ohe, **unbound_file_ohe}
    if not combined_data:
        print(f"Warning: No combined OHE data for {tf} - {cell_line}.")
        return np.array([]), np.array([])

    # Process additional features if present
    if len(features) > 1 and DNAflexpy:
        for feature in features[1:]:  # Skip "1mer"
            print(f"  Calculating flexibility with {feature}...")
            try:
                # Run DNAflexpy for bound and unbound separately
                # Check file existence before calling DNAflexpy
                bound_flex_df = pd.DataFrame()
                if os.path.exists(bound_file):
                    bound_flex_df = DNAflexpy(input_file=bound_file, window_size=0, feature=feature, threads=8) if bound_file_ohe else pd.DataFrame()

                unbound_flex_df = pd.DataFrame()
                if os.path.exists(unbound_file):
                     # Get original unbound IDs before resampling for filtering DNAflexpy output
                     original_unbound_ohe = one_hot_encode_fasta(unbound_file, label=0)
                     unbound_flex_df = DNAflexpy(input_file=unbound_file, window_size=0, feature=feature, threads=8) if original_unbound_ohe else pd.DataFrame()


                # Filter unbound DF to only include the *resampled* sequences
                if not unbound_flex_df.empty and len(unbound_file_ohe) < len(original_unbound_ohe): # Check if resampling occurred
                    unbound_ids_to_keep = list(unbound_file_ohe.keys())
                    # Assuming the first column (index 0) of flex_df contains sequence IDs
                    # Ensure the ID column exists before filtering
                    if unbound_flex_df.columns[0] in unbound_flex_df.columns:
                        unbound_flex_df = unbound_flex_df[unbound_flex_df.iloc[:, 0].astype(str).isin(unbound_ids_to_keep)]
                    else:
                        print(f"  Warning: ID column '{unbound_flex_df.columns[0]}' not found in unbound flex data for {feature}. Cannot filter.")
                        unbound_flex_df = pd.DataFrame() # Clear df if cannot filter


                # Combine bound and (filtered) unbound flex data
                if not bound_flex_df.empty or not unbound_flex_df.empty:
                     flex_df = pd.concat([bound_flex_df, unbound_flex_df]).reset_index(drop=True)
                else:
                     print(f"  Warning: No flexibility data generated or kept for feature {feature}.")
                     continue # Skip this feature if no data

                # --- START: Handle Duplicate IDs in flex_df ---
                if not flex_df.empty:
                    id_col_name = flex_df.columns[0] # Assume first column is ID
                    # Check if the assumed ID column actually exists
                    if id_col_name not in flex_df.columns:
                         print(f"  Error: Assumed ID column '{id_col_name}' not found in flex_df for feature '{feature}'. Skipping.")
                         continue
                    num_duplicates = flex_df.duplicated(subset=[id_col_name]).sum()
                    if num_duplicates > 0:
                        print(f"    Warning: Found {num_duplicates} duplicate sequence IDs in flexibility data for feature '{feature}'. Keeping first occurrence.")
                        flex_df = flex_df.drop_duplicates(subset=[id_col_name], keep='first').reset_index(drop=True)
                # --- END: Handle Duplicate IDs in flex_df ---

                if flex_df.empty: # Check again after potential dropping
                    print(f"  Warning: Flexibility DataFrame became empty after handling duplicates for feature {feature}.")
                    continue

                flex_df.iloc[:, 0] = flex_df.iloc[:, 0].astype(str) # Ensure ID column is string
                # Now, setting index and converting to dict should not warn about unique columns
                flex_df_dict = flex_df.set_index(id_col_name).T.to_dict('list')

            except Exception as e:
                print(f"  Error calculating or processing feature {feature}: {e}")
                traceback.print_exc() # Print traceback for debugging flex issues
                continue # Skip this feature on error

            # Add flexibility features to combined_data
            feature_length_flex = 0
            if flex_df_dict:
                 # Get expected length of flex features + interaction terms
                 first_flex_key = next(iter(flex_df_dict))
                 first_flex_vals = flex_df_dict[first_flex_key]
                 feature_length_flex = len(first_flex_vals) # Scaled
                 if len(first_flex_vals) > 1:
                     feature_length_flex += (len(first_flex_vals) - 1) # Interaction terms

            for seq_id, data in combined_data.items():
                # Ensure 'feat' exists from OHE
                if 'feat' not in data:
                    print(f"  Warning: OHE feature missing for {seq_id}. Cannot add flex features.")
                    continue

                flex_values = flex_df_dict.get(str(seq_id)) # Ensure seq_id is string for lookup
                if flex_values:
                    flex_feat = np.array(flex_values, dtype=np.float32)

                    # Simple Scaling (MinMax to 0-1 range)
                    min_val, max_val = np.min(flex_feat), np.max(flex_feat)
                    scaled_flex_feat = np.zeros_like(flex_feat) # Default to zeros
                    if max_val > min_val:
                        scaled_flex_feat = (flex_feat - min_val) / (max_val - min_val)

                    # Interaction Terms
                    scaled_interaction_terms = np.array([], dtype=np.float32)
                    if len(scaled_flex_feat) > 1:
                        interaction_terms = scaled_flex_feat[:-1] * scaled_flex_feat[1:]
                        min_int, max_int = np.min(interaction_terms), np.max(interaction_terms)
                        if max_int > min_int:
                            scaled_interaction_terms = (interaction_terms - min_int) / (max_int - min_int)
                        else:
                            scaled_interaction_terms = np.zeros_like(interaction_terms) # Handle constant interaction

                    # Concatenate features
                    data['feat'] = np.concatenate([data['feat'],
                                                   scaled_flex_feat,
                                                   scaled_interaction_terms])
                else:
                    # If flex data is missing for this ID, pad with zeros to maintain consistent length
                    # print(f"  Debug: Padding missing flex data for {seq_id}, feature {feature}")
                    if feature_length_flex > 0:
                         padding = np.zeros(feature_length_flex, dtype=np.float32)
                         data['feat'] = np.concatenate([data['feat'], padding])
                    # else: # No flex data was generated at all for this feature, length is 0, no padding needed


    # Prepare feature matrix (X) and target vector (y)
    # Ensure all feature vectors have the same length before creating the matrix
    if not combined_data:
        print("No combined data available before final processing.")
        return np.array([]), np.array([])

    # Determine expected feature length from the first valid entry
    expected_feature_length = None
    valid_data = {}
    processed_ids = set() # Keep track of IDs added to valid_data

    # Find expected length from the first item that has features
    for seq_id, data in combined_data.items():
        if 'feat' in data:
            expected_feature_length = len(data['feat'])
            # print(f"Debug: Determined expected feature length: {expected_feature_length} from seq_id {seq_id}")
            break

    if expected_feature_length is None:
        print("Error: Could not determine expected feature length. No sequences have features.")
        return np.array([]), np.array([])

    # Filter based on the determined length and ensure uniqueness
    for seq_id, data in combined_data.items():
         current_len = len(data.get('feat', []))
         if current_len == expected_feature_length:
             if seq_id not in processed_ids: # Ensure we don't add duplicates from combined_data itself
                 valid_data[seq_id] = data
                 processed_ids.add(seq_id)
         # else:
             # print(f"Debug: Skipping seq_id {seq_id}. Feature length {current_len} != {expected_feature_length}")


    num_removed = len(combined_data) - len(valid_data)
    if num_removed > 0:
        print(f"Warning: {num_removed} sequences removed due to inconsistent feature lengths or missing features.")

    if not valid_data:
        print("No valid data remaining after feature extraction and length check.")
        return np.array([]), np.array([])

    # Create X and y from the filtered valid_data
    # Ensure consistent order by sorting keys (optional but good practice)
    sorted_ids = sorted(valid_data.keys())
    try:
        X = np.array([valid_data[seq_id]['feat'] for seq_id in sorted_ids], dtype=np.float32)
        y = np.array([valid_data[seq_id]['label'] for seq_id in sorted_ids], dtype=np.int32)
    except KeyError as e:
         print(f"Error creating final arrays: Missing key {e}. Check data consistency.")
         return np.array([]), np.array([])
    except ValueError as e:
         print(f"Error creating final arrays: {e}. Likely due to inconsistent sub-array shapes.")
         # Add more debug prints here if needed, e.g., print lengths of individual 'feat' arrays
         # for seq_id in sorted_ids:
         #     print(f"ID: {seq_id}, Length: {len(valid_data[seq_id]['feat'])}")
         return np.array([]), np.array([])


    print(f"Final dataset shape for {tf} - {cell_line}: X={X.shape}, y={y.shape}")

    # Basic sanity check
    if X.shape[0] == 0 or X.shape[0] != y.shape[0]:
        print(f"CRITICAL WARNING: Mismatch or empty arrays. X samples ({X.shape[0]}) vs y labels ({y.shape[0]}) for {tf}-{cell_line}")
        return np.array([]), np.array([])

    return X, y



# def train_and_evaluate_with_nested_cv(tf, cell_line, features=('1mer',)):
#     """Trains and evaluates XGBoost using nested CV with GPU, avoiding device mismatch issues during prediction."""
#     X, y = extract_and_combine_features(tf, cell_line, features)

#     if X.shape[0] < 10 or y.shape[0] < 10 or X.ndim != 2 or X.shape[0] != y.shape[0]:
#         print(f"Skipping {tf} in {cell_line} with features {features} due to insufficient/invalid data (X:{X.shape}, y:{y.shape})")
#         return []  # Return empty list, indicating no results for this combo

#     # --- XGBoost Classifier Initialization ---
#     pipeline = Pipeline([
#         ('classifier', XGBClassifier(
#             objective='binary:logistic',
#             eval_metric='logloss',
#             device='cuda',
#             random_state=42
#         ))
#     ])

#     param_grid = {
#         'classifier__n_estimators': [50, 100, 200, 300],
#         'classifier__max_depth': [3, 5, 7, 10],
#         'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3],
#     }

#     inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
#     outer_cv = StratifiedKFold(n_splits=TARGET_FOLDS, shuffle=True, random_state=42)

#     grid_search = GridSearchCV(
#         estimator=pipeline,
#         param_grid=param_grid,
#         cv=inner_cv,
#         scoring='roc_auc',
#         n_jobs=1,
#         verbose=0,
#         error_score='raise'
#     )

#     fold_results = []
#     features_str = "_".join(features)

#     print(f"\nStarting Nested CV for {tf} - {cell_line} with features {features_str}")
#     outer_cv_iterable = tqdm(outer_cv.split(X, y),
#                              total=outer_cv.get_n_splits(),
#                              desc=f"Outer CV {tf}-{cell_line} ({features_str})")

#     for fold, (train_idx, test_idx) in enumerate(outer_cv_iterable, 1):
#         try:
#             X_train, X_test = X[train_idx], X[test_idx]
#             y_train, y_test = y[train_idx], y[test_idx]

#             if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
#                 print(f"  Fold {fold}: Skipping fold due to lack of class diversity in train/test split.")
#                 auc_roc, mcc = np.nan, np.nan
#                 best_params_str = "Skipped - Class diversity issue"
#             else:
#                 grid_search.fit(X_train, y_train)

#                 # --- Get trained classifier only (not pipeline) ---
#                 classifier = grid_search.best_estimator_.named_steps['classifier']

#                 # --- Ensure test data is on GPU ---
#                 if cp:
#                     try:
#                         ### !! IMPORTANT .get() is required to get into gpu !! ###
#                         X_test_gpu = cp.asarray(X_test).get() 
#                         print(f"  Fold {fold}: Prediction on device: {X_test_gpu.device}")
#                     except Exception as e:
#                         print(f"  Fold {fold}: Warning - CuPy conversion failed: {e}. Falling back to CPU.")
#                         X_test_gpu = X_test
#                 else:
#                     X_test_gpu = X_test

#                 # --- Predict ---
#                 y_pred_proba = classifier.predict_proba(X_test_gpu)[:, 1]
#                 y_pred = classifier.predict(X_test_gpu)

#                 # --- Metric Calculation ---
#                 y_test_np = y_test  # y_test is already on CPU
#                 y_pred_proba_np = cp.asnumpy(y_pred_proba) if cp and isinstance(y_pred_proba, cp.ndarray) else y_pred_proba
#                 y_pred_np = cp.asnumpy(y_pred) if cp and isinstance(y_pred, cp.ndarray) else y_pred

#                 try:
#                     auc_roc = roc_auc_score(y_test_np, y_pred_proba_np)
#                     mcc = matthews_corrcoef(y_test_np, y_pred_np)
#                     best_params_str = str(grid_search.best_params_)
#                 except ValueError as metric_ve:
#                     print(f"  Fold {fold}: Metric calculation error: {metric_ve}")
#                     auc_roc, mcc = np.nan, np.nan
#                     best_params_str = f"Metrics Error: {metric_ve}"
#                 except Exception as metric_e:
#                     print(f"  Fold {fold}: Unexpected error during metric calculation: {metric_e}")
#                     auc_roc, mcc = np.nan, np.nan
#                     best_params_str = f"Metrics Error: {metric_e}"

#         except Exception as fold_e:
#             print(f"\n!!!!!!!! ERROR during Fold {fold} for {tf}-{cell_line} ({features_str}) !!!!!!!!!!")
#             traceback.print_exc()
#             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#             auc_roc, mcc = np.nan, np.nan
#             best_params_str = f"Fold Error: {fold_e}"

#         # --- Record Fold Results ---
#         result = {
#             'tf': tf, 'cell_line': cell_line, 'fold': fold, 'auc_roc': auc_roc,
#             'mcc': mcc, 'features': features_str, 'best_params': best_params_str
#         }
#         fold_results.append(result)

#         # --- Save to JSON Lines File ---
#         try:
#             serializable_result = result.copy()
#             serializable_result['auc_roc'] = None if np.isnan(auc_roc) else float(auc_roc)
#             serializable_result['mcc'] = None if np.isnan(mcc) else float(mcc)

#             with open(JSON_FULL_PATH, 'a') as f_json:
#                 json.dump(serializable_result, f_json)
#                 f_json.write('\n')
#         except Exception as json_e:
#             print(f"  Fold {fold}: Warning - Failed to write result to JSON file {JSON_OUTPUT_FILENAME}: {json_e}")

#     # --- Summary ---
#     auc_roc_scores = [r['auc_roc'] for r in fold_results if r['auc_roc'] is not None and not np.isnan(r['auc_roc'])]
#     mcc_scores = [r['mcc'] for r in fold_results if r['mcc'] is not None and not np.isnan(r['mcc'])]

#     print(f"\nFinished Nested CV for {tf} - {cell_line} with features {features_str}")
#     if auc_roc_scores:
#         avg_auc = np.mean(auc_roc_scores)
#         std_auc = np.std(auc_roc_scores)
#         avg_mcc = np.mean(mcc_scores)
#         std_mcc = np.std(mcc_scores)
#         print(f"  Average AUC-ROC: {avg_auc:.4f} ± {std_auc:.4f} ({len(auc_roc_scores)} valid folds)")
#         print(f"  Average MCC    : {avg_mcc:.4f} ± {std_mcc:.4f} ({len(mcc_scores)} valid folds)")
#     else:
#         print(f"  No valid scores obtained across {len(fold_results)} folds.")

#     # --- Clean up GPU memory ---
#     if cp and 'X_test_gpu' in locals() and isinstance(X_test_gpu, cp.ndarray):
#         del X_test_gpu
#         cp.get_default_memory_pool().free_all_blocks()

#     return fold_results

# def train_and_evaluate_with_nested_cv(tf, cell_line, features=('1mer',)):
#     """Trains and evaluates XGBoost using nested CV with GPU, avoiding device mismatch issues during prediction."""
#     X, y = extract_and_combine_features(tf, cell_line, features)

#     if X.shape[0] < 10 or y.shape[0] < 10 or X.ndim != 2 or X.shape[0] != y.shape[0]:
#         print(f"Skipping {tf} in {cell_line} with features {features} due to insufficient/invalid data (X:{X.shape}, y:{y.shape})")
#         return []  # Return empty list, indicating no results for this combo

#     # --- XGBoost Parameter Initialization ---
#     param_grid = {
#         'n_estimators': [50, 100, 200, 300],
#         'max_depth': [3, 5, 7, 10],
#         'learning_rate': [0.01, 0.1, 0.2, 0.3],
#     }

#     inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
#     outer_cv = StratifiedKFold(n_splits=TARGET_FOLDS, shuffle=True, random_state=42)

#     fold_results = []
#     features_str = "_".join(features)

#     print(f"\nStarting Nested CV for {tf} - {cell_line} with features {features_str}")
#     outer_cv_iterable = tqdm(outer_cv.split(X, y),
#                              total=outer_cv.get_n_splits(),
#                              desc=f"Outer CV {tf}-{cell_line} ({features_str})")

#     for fold, (train_idx, test_idx) in enumerate(outer_cv_iterable, 1):
#         try:
#             # --- Train/Test Split ---
#             X_train, X_test = X[train_idx], X[test_idx]
#             y_train, y_test = y[train_idx], y[test_idx]

#             # --- Ensure data is on GPU ---
#             if cp:
#                 try:
#                     # Move data to GPU
#                     X_train_gpu = cp.asarray(X_train)
#                     X_test_gpu = cp.asarray(X_test)
#                     y_train_gpu = cp.asarray(y_train)
#                     y_test_gpu = cp.asarray(y_test)

#                     # Create DMatrix for XGBoost
#                     dtrain = xgb.DMatrix(X_train_gpu, label=y_train_gpu)
#                     dtest = xgb.DMatrix(X_test_gpu, label=y_test_gpu)
#                     print(f"  Fold {fold}: Data moved to GPU successfully.")
#                 except Exception as e:
#                     print(f"  Fold {fold}: Warning - CuPy conversion failed: {e}. Skipping this fold.")
#                     continue  # Skip this fold if GPU conversion fails
#             else:
#                 print(f"  Fold {fold}: CuPy not available. Skipping GPU conversion.")
#                 continue  # Skip this fold if GPU is not available

#             if len(cp.unique(y_train_gpu)) < 2 or len(cp.unique(y_test_gpu)) < 2:
#                 print(f"  Fold {fold}: Skipping fold due to lack of class diversity in train/test split.")
#                 auc_roc, mcc = np.nan, np.nan
#                 best_params_str = "Skipped - Class diversity issue"
#             else:
#                 best_auc = 0
#                 best_params = None

#                 # --- Perform Grid Search Manually ---
#                 for n_estimators in param_grid['n_estimators']:
#                     for max_depth in param_grid['max_depth']:
#                         for learning_rate in param_grid['learning_rate']:
#                             params = {
#                                 'objective': 'binary:logistic',
#                                 'eval_metric': 'logloss',
#                                 'tree_method': 'gpu_hist',
#                                 'n_estimators': n_estimators,
#                                 'max_depth': max_depth,
#                                 'learning_rate': learning_rate,
#                                 'random_state': 42,
#                             }

#                             model = XGBClassifier.train(params, dtrain, num_boost_round=n_estimators)
#                             y_pred_proba = model.predict(dtest)
#                             y_pred = (y_pred_proba > 0.5).astype(int)

#                             # --- Metric Calculation ---
#                             y_test_np = cp.asnumpy(y_test_gpu)
#                             y_pred_proba_np = cp.asnumpy(y_pred_proba)
#                             y_pred_np = cp.asnumpy(y_pred)

#                             try:
#                                 auc_roc = roc_auc_score(y_test_np, y_pred_proba_np)
#                                 mcc = matthews_corrcoef(y_test_np, y_pred_np)
#                                 if auc_roc > best_auc:
#                                     best_auc = auc_roc
#                                     best_params = params
#                             except ValueError as metric_ve:
#                                 print(f"  Fold {fold}: Metric calculation error: {metric_ve}")
#                                 continue
#                             except Exception as metric_e:
#                                 print(f"  Fold {fold}: Unexpected error during metric calculation: {metric_e}")
#                                 continue

#                 auc_roc = best_auc
#                 best_params_str = str(best_params) if best_params else "No valid parameters found"

#         except Exception as fold_e:
#             print(f"\n!!!!!!!! ERROR during Fold {fold} for {tf}-{cell_line} ({features_str}) !!!!!!!!!!")
#             traceback.print_exc()
#             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#             auc_roc, mcc = np.nan, np.nan
#             best_params_str = f"Fold Error: {fold_e}"

#         # --- Record Fold Results ---
#         result = {
#             'tf': tf, 'cell_line': cell_line, 'fold': fold, 'auc_roc': auc_roc,
#             'mcc': mcc, 'features': features_str, 'best_params': best_params_str
#         }
#         fold_results.append(result)

#         # --- Save to JSON Lines File ---
#         try:
#             serializable_result = result.copy()
#             serializable_result['auc_roc'] = None if np.isnan(auc_roc) else float(auc_roc)
#             serializable_result['mcc'] = None if np.isnan(mcc) else float(mcc)

#             with open(JSON_FULL_PATH, 'a') as f_json:
#                 json.dump(serializable_result, f_json)
#                 f_json.write('\n')
#         except Exception as json_e:
#             print(f"  Fold {fold}: Warning - Failed to write result to JSON file {JSON_OUTPUT_FILENAME}: {json_e}")

#     # --- Summary ---
#     auc_roc_scores = [r['auc_roc'] for r in fold_results if r['auc_roc'] is not None and not np.isnan(r['auc_roc'])]
#     mcc_scores = [r['mcc'] for r in fold_results if r['mcc'] is not None and not np.isnan(r['mcc'])]

#     print(f"\nFinished Nested CV for {tf} - {cell_line} with features {features_str}")
#     if auc_roc_scores:
#         avg_auc = np.mean(auc_roc_scores)
#         std_auc = np.std(auc_roc_scores)
#         avg_mcc = np.mean(mcc_scores)
#         std_mcc = np.std(mcc_scores)
#         print(f"  Average AUC-ROC: {avg_auc:.4f} ± {std_auc:.4f} ({len(auc_roc_scores)} valid folds)")
#         print(f"  Average MCC    : {avg_mcc:.4f} ± {std_mcc:.4f} ({len(mcc_scores)} valid folds)")
#     else:
#         print(f"  No valid scores obtained across {len(fold_results)} folds.")

#     # --- Clean up GPU memory ---
#     if cp:
#         cp.get_default_memory_pool().free_all_blocks()

#     return fold_results
    
# --- MAIN EXECUTION ---

# Load summary of already completed folds from JSON log
completed_folds_summary = load_completed_folds(JSON_FULL_PATH)

# Feature sets to evaluate
feature_sets = [ ("1mer", ),
    ("1mer", "DNaseI", "NPP", "twistDisp", "trx", "stiffness")
    ]

# Store all results generated *in this run*
current_run_results = []

# List of TF and cell line pairs to process
tf_cell_line_pairs = []
metadata_file = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/ENCODE_CLF.meta" # Ensure this path is correct
if os.path.exists(metadata_file):
    print(f"Reading metadata from: {metadata_file}")
    try:
        with open(metadata_file) as meta:
            for line in meta:
                rec = line.strip()
                if not rec or rec.startswith('#'): continue # Skip empty/comment lines
                parts = rec.split('_')
                if len(parts) >= 2:
                    tf = parts[0]
                    cline = parts[1]
                    pair = (tf, cline)
                    if pair not in tf_cell_line_pairs:
                        tf_cell_line_pairs.append(pair)
                else:
                    print(f"Skipping malformed line in metadata: {rec}")
    except Exception as meta_e:
         print(f"Error reading metadata file {metadata_file}: {meta_e}")

else:
    print(f"Error: Metadata file not found at {metadata_file}")
    # Optionally define some pairs manually for testing if metadata is missing
    # tf_cell_line_pairs = [('CTCF', 'HEPG2'), ('EGR1', 'K562')] # Example

print(f"Found {len(tf_cell_line_pairs)} TF-Cell Line pairs to process.")
print(f"Processing {len(feature_sets)} feature sets for each pair.")
print(f"Target folds per combination: {TARGET_FOLDS}")
# Note: JSON file is appended to, not cleared, to allow resuming.

# Main loop
if tf_cell_line_pairs:
    total_combinations = len(tf_cell_line_pairs) * len(feature_sets)
    processed_count = 0
    skipped_count = 0

    for tf, cell_line in tf_cell_line_pairs:
        for features in feature_sets:
            processed_count += 1
            features_str = "_".join(features)
            current_key = (tf, cell_line, features_str)

            # --- START: Check if combination is already complete --- <<< MODIFICATION
            completed_count = completed_folds_summary.get(current_key, 0)
            if completed_count >= TARGET_FOLDS:
                print(f"\n--- ({processed_count}/{total_combinations}) SKIPPING {tf} - {cell_line} ({features_str}) ---")
                print(f"    Reason: Found {completed_count} completed folds in log (>= target {TARGET_FOLDS}).")
                skipped_count += 1
                continue # Skip to the next feature set or TF/cell line pair
            else:
                 print(f"\n===== ({processed_count}/{total_combinations}) Processing {tf} - {cell_line} ({features_str}) =====")
                 if completed_count > 0:
                     print(f"      (Found {completed_count} existing folds in log, resuming...)")
            # --- END: Check if combination is already complete ---

            try:
                # Run the nested CV for the current combination
                # JSON writing happens inside this function now
                results = train_and_evaluate_with_nested_cv(tf, cell_line, features)
                current_run_results.extend(results) # Add results from this run

            except Exception as e:
                # Log the error and continue with the next pair/feature set
                print(f"\n!!!!!!!! CRITICAL ERROR processing {tf} in {cell_line} with features {features_str} !!!!!!!!!!")
                print(f"Error type: {type(e).__name__}")
                print(f"Error message: {e}")
                traceback.print_exc() # Print full traceback for debugging
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                # Optionally add a placeholder result indicating failure for the CSV
                # We don't write this critical error to JSON here, only fold-level results/errors
                error_result = {
                    'tf': tf, 'cell_line': cell_line, 'fold': -1, 'auc_roc': np.nan,
                    'mcc': np.nan, 'features': features_str, 'best_params': f"Critical Error: {e}"
                }
                current_run_results.append(error_result)
                # No JSON write here for the critical error itself


# --- Save Final Aggregated Results (Original CSV Logic - Now uses current_run_results) ---
if current_run_results:
    results_df = pd.DataFrame(current_run_results)
    # Ensure float precision in CSV output
    output_filename = "nested_cv_gpu_results_THIS_RUN.csv" # Rename to avoid overwriting full history easily
    print(f"\n--- Saving results from THIS RUN ({len(current_run_results)} rows) to {output_filename} ---")
    print(results_df.head())
    try:
        # Convert NaN to empty string or specific marker for CSV if desired
        results_df.fillna({'auc_roc': 'NaN', 'mcc': 'NaN'}, inplace=True) # Or use float_format in to_csv
        results_df.to_csv(output_filename, index=False, float_format='%.6f')
        print("Aggregated CSV results for this run saved.")
    except Exception as csv_e:
        print(f"Error saving aggregated results to CSV: {csv_e}")
else:
    print("\nNo new results were generated in this run (CSV).")

print(f"\nTotal combinations processed/skipped in this run: {processed_count - skipped_count}/{skipped_count}")
print(f"Iterative fold results (including previous runs) are in {JSON_FULL_PATH}")
print("\nScript finished.")


cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.
Attempted to load cuml.accel extension (effective in IPython/Jupyter).
CuPy imported successfully.
Reading existing results from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/XGB_ENCODE_RESULTS/nested_cv_fold_results.jsonl...
Found 151 combinations with existing results.
Reading metadata from: /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/ENCODE_CLF.meta
Found 487 TF-Cell Line pairs to process.
Processing 2 feature sets for each pair.
Target folds per combination: 10

--- (1/974) SKIPPING ATF2 - GM12878 (1mer) ---
    Reason: Found 13 completed folds in log (>= target 10).

--- (2/974) SKIPPING ATF2 - GM12878 (1mer_DNaseI_NPP_twistDisp_trx_stiffness) ---
    Reason: Found 10 completed folds in log (>= target 10).

--- (3/974) SKIPPING ATF2 - H1 (1mer) ---
    Reason: Found 10 completed folds in log (>= target 10).

--- (4/974) SKIPPING ATF2 - H1 (1mer_DNaseI_NPP_twistDisp_t

Traceback (most recent call last):
  File "/tmp/ipykernel_56550/117345688.py", line 752, in <module>
    results = train_and_evaluate_with_nested_cv(tf, cell_line, features)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'train_and_evaluate_with_nested_cv' is not defined
Traceback (most recent call last):
  File "/tmp/ipykernel_56550/117345688.py", line 752, in <module>
    results = train_and_evaluate_with_nested_cv(tf, cell_line, features)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'train_and_evaluate_with_nested_cv' is not defined
Traceback (most recent call last):
  File "/tmp/ipykernel_56550/117345688.py", line 752, in <module>
    results = train_and_evaluate_with_nested_cv(tf, cell_line, features)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'train_and_evaluate_with_nested_cv' is not defined
Traceback (most recent call last):
  File "/tmp/ipykernel_56550/117345688.py", line 752, in <module>
    results = train_and_evaluate

### 2. Install additional dependencies
    i. Biopython
    ii. DNAflexpy python package

In [16]:
# !pip install Bio
# ! pip install ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension
# !pip install git+https://github.com/UpalabdhaD/DNAflexpy.git

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


### Import packages
    i. rapids library for GPU accelaration
    ii. sklearn for modelling
    iii. DNaflexpy for feature transformation

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py --prerelease if-necessary

# Commented out IPython magic to ensure Python compatibility.
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
import json # <<< Added for JSON output
import traceback # For printing full tracebacks on error

# --- GPU ACCELERATION SETUP ---
# Attempt to automatically patch scikit-learn functions with cuML equivalents
# This might accelerate GridSearchCV, StratifiedKFold, metrics etc. if compatible versions exist.
try:
    %load_ext cuml.accel
    # If running as a script, explicit patching might be needed if desired,
    # but XGBoost GPU usage is handled separately.
    print("Attempted to load cuml.accel extension (effective in IPython/Jupyter).")
except Exception as e:
    print(f"Could not load cuml.accel: {e}. Proceeding without automatic patching.")
    print("XGBoost GPU acceleration will still be attempted if configured.")
# -----------------------------

# Scikit-learn imports (might be patched by cuml.accel)
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, matthews_corrcoef
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

# Import XGBoost separately
from xgboost import XGBClassifier

# Import tqdm for progress bars
from tqdm.auto import tqdm

# Import cupy for potential explicit data conversion if needed
try:
    import cupy as cp
    print("CuPy imported successfully.")
except ImportError:
    print("CuPy not found. Explicit GPU array conversion will not be possible.")
    # cp = None # Define cp as None if not available

# Assuming DNAflexpy is available in the environment
try:
    from DNAflexpy.core import DNAflexpyMP as DNAflexpy
except ImportError:
    print("Warning: DNAflexpy not found. Feature extraction requiring it will fail.")
    # Define a placeholder or handle the error as appropriate
    DNAflexpy = None

# Define the path to the directory containing the FASTA files
# Adjust this path as necessary
# data_dir = "../../RESULTS/INVIVO_ENCODE"
# Use the provided path for Colab example
data_dir='/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/XGB_ENCODE_RESULTS' # <<< Updated path

if not os.path.isdir(data_dir):
    print(f"Warning: Data directory not found at {data_dir}")

# Define the JSON output file name <<< Added
JSON_BASE_FILENAME = "nested_cv_fold_results.jsonl"
JSON_FULL_PATH = os.path.join(data_dir, JSON_BASE_FILENAME)
TARGET_FOLDS = 10 # <<< Define the expected number of folds for completion


cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.
Attempted to load cuml.accel extension (effective in IPython/Jupyter).
CuPy imported successfully.


### 2. Functions

In [2]:

# --- START: Function to load completed folds --- <<< NEW FUNCTION
def load_completed_folds(json_filename):
    """Reads the JSON Lines log file and counts completed folds per combination."""
    completed_counts = {}
    if not os.path.exists(json_filename):
        print(f"Log file {json_filename} not found. Starting fresh.")
        return completed_counts

    print(f"Reading existing results from {json_filename}...")
    with open(json_filename, 'r') as f_json:
        for line_num, line in enumerate(f_json):
            line = line.strip()
            if not line:
                continue # Skip empty lines
            try:
                record = json.loads(line)
                # Ensure necessary keys exist (handle potential malformed lines)
                if all(k in record for k in ['tf', 'cell_line', 'features', 'fold']):
                    # Use the same feature string format as stored in JSON
                    features_str = record['features']
                    key = (record['tf'], record['cell_line'], features_str)
                    completed_counts[key] = completed_counts.get(key, 0) + 1
                else:
                    print(f"  Warning: Skipping malformed record on line {line_num+1} (missing keys).")
            except json.JSONDecodeError:
                print(f"  Warning: Skipping invalid JSON on line {line_num+1}: {line[:100]}...") # Print start of bad line
            except Exception as e:
                 print(f"  Warning: Unexpected error processing line {line_num+1}: {e}")

    print(f"Found {len(completed_counts)} combinations with existing results.")
    return completed_counts

# --- END: Function to load completed folds ---

def manual_one_hot_encoding(seq, alphabet=['A', 'C', 'G', 'T']):
    """Manually one-hot encodes a DNA sequence."""
    mapping = {nt: i for i, nt in enumerate(alphabet)}
    one_hot = np.zeros((len(seq), len(alphabet)), dtype=np.float32) # Use float32 for GPU compatibility
    for i, nt in enumerate(seq):
        nt_upper = nt.upper() # Ensure case-insensitivity
        if nt_upper in mapping:
            one_hot[i, mapping[nt_upper]] = 1.0
    return one_hot.flatten()

def one_hot_encode_fasta(fa_filepath, label=1, alphabet=['A', 'C', 'G', 'T']):
    """
    One-hot encodes DNA sequences from a FASTA file.
    Returns a dictionary {seq_id: {'id': seq_id, 'feat': np.array, 'label': label}}.
    """
    encoded_data = {}
    if not os.path.exists(fa_filepath):
        print(f"Warning: FASTA file not found: {fa_filepath}")
        return encoded_data

    try:
        with open(fa_filepath, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                seq_id = record.id
                seq = str(record.seq)

                # One-hot encode the sequence
                one_hot_features = manual_one_hot_encoding(seq, alphabet)

                # Store in the dictionary
                encoded_data[seq_id] = {'id': seq_id,
                                        'feat': one_hot_features,
                                        'label': label}
    except Exception as e:
        print(f"Error reading or parsing FASTA file {fa_filepath}: {e}")
        return {} # Return empty dict on error
    return encoded_data

def extract_and_combine_features(tf, cell_line, features, alphabet=['A', 'C', 'G', 'T']):
    """
    Extracts features (starting with one-hot encoding) and combines them.
    Handles duplicate sequence IDs in flexibility data by keeping the first occurrence.
    Returns NumPy arrays X, y.
    """
    print(f"Processing {tf} - {cell_line}...")
    # Construct full paths within the function
    sequences_dir = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/INVIVO_ENCODE/SEQUENCES"
    bound_file = os.path.join(sequences_dir, f"{tf}_{cell_line}_BOUND_20.fa")
    unbound_file = os.path.join(sequences_dir, f"{tf}_{cell_line}_UNBOUND_20.fa")

    # Check if SEQUENCES directory exists
    if not os.path.isdir(sequences_dir):
         print(f"Error: SEQUENCES directory not found at {sequences_dir}")
         return np.array([]), np.array([])

    # One-hot encode the sequences
    bound_file_ohe = one_hot_encode_fasta(bound_file, label=1, alphabet=alphabet)
    unbound_file_ohe = one_hot_encode_fasta(unbound_file, label=0, alphabet=alphabet)

    if not bound_file_ohe or not unbound_file_ohe:
        print(f"Warning: Missing or empty bound/unbound OHE data for {tf} - {cell_line}. Skipping feature extraction.")
        return np.array([]), np.array([]) # Return empty arrays

    # Sample equal number of unbound data
    bound_size = len(bound_file_ohe)
    # Ensure unbound_file_ohe is not empty before checking length
    if unbound_file_ohe and len(unbound_file_ohe) < bound_size:
         print(f"Warning: Fewer unbound samples ({len(unbound_file_ohe)}) than bound samples ({bound_size}). Using all unbound.")
         # No resampling needed if unbound is smaller
    elif unbound_file_ohe: # Only resample if unbound_file_ohe is not empty and >= bound_size
         unbound_items = list(unbound_file_ohe.items())
         unbound_file_ohe_items = resample(unbound_items,
                                        n_samples=bound_size,
                                        replace=False, # No replacement if unbound >= bound
                                        random_state=42)
         unbound_file_ohe = dict(unbound_file_ohe_items)
    # else: unbound_file_ohe is empty, handled by the initial check


    combined_data = {**bound_file_ohe, **unbound_file_ohe}
    if not combined_data:
        print(f"Warning: No combined OHE data for {tf} - {cell_line}.")
        return np.array([]), np.array([])

    # Process additional features if present
    if len(features) > 1 and DNAflexpy:
        for feature in features[1:]:  # Skip "1mer"
            print(f"  Calculating flexibility with {feature}...")
            try:
                # Run DNAflexpy for bound and unbound separately
                # Check file existence before calling DNAflexpy
                bound_flex_df = pd.DataFrame()
                if os.path.exists(bound_file):
                    bound_flex_df = DNAflexpy(input_file=bound_file, window_size=0, feature=feature, threads=8) if bound_file_ohe else pd.DataFrame()

                unbound_flex_df = pd.DataFrame()
                if os.path.exists(unbound_file):
                     # Get original unbound IDs before resampling for filtering DNAflexpy output
                     original_unbound_ohe = one_hot_encode_fasta(unbound_file, label=0)
                     unbound_flex_df = DNAflexpy(input_file=unbound_file, window_size=0, feature=feature, threads=8) if original_unbound_ohe else pd.DataFrame()


                # Filter unbound DF to only include the *resampled* sequences
                if not unbound_flex_df.empty and len(unbound_file_ohe) < len(original_unbound_ohe): # Check if resampling occurred
                    unbound_ids_to_keep = list(unbound_file_ohe.keys())
                    # Assuming the first column (index 0) of flex_df contains sequence IDs
                    # Ensure the ID column exists before filtering
                    if unbound_flex_df.columns[0] in unbound_flex_df.columns:
                        unbound_flex_df = unbound_flex_df[unbound_flex_df.iloc[:, 0].astype(str).isin(unbound_ids_to_keep)]
                    else:
                        print(f"  Warning: ID column '{unbound_flex_df.columns[0]}' not found in unbound flex data for {feature}. Cannot filter.")
                        unbound_flex_df = pd.DataFrame() # Clear df if cannot filter


                # Combine bound and (filtered) unbound flex data
                if not bound_flex_df.empty or not unbound_flex_df.empty:
                     flex_df = pd.concat([bound_flex_df, unbound_flex_df]).reset_index(drop=True)
                else:
                     print(f"  Warning: No flexibility data generated or kept for feature {feature}.")
                     continue # Skip this feature if no data

                # --- START: Handle Duplicate IDs in flex_df ---
                if not flex_df.empty:
                    id_col_name = flex_df.columns[0] # Assume first column is ID
                    # Check if the assumed ID column actually exists
                    if id_col_name not in flex_df.columns:
                         print(f"  Error: Assumed ID column '{id_col_name}' not found in flex_df for feature '{feature}'. Skipping.")
                         continue
                    num_duplicates = flex_df.duplicated(subset=[id_col_name]).sum()
                    if num_duplicates > 0:
                        print(f"    Warning: Found {num_duplicates} duplicate sequence IDs in flexibility data for feature '{feature}'. Keeping first occurrence.")
                        flex_df = flex_df.drop_duplicates(subset=[id_col_name], keep='first').reset_index(drop=True)
                # --- END: Handle Duplicate IDs in flex_df ---

                if flex_df.empty: # Check again after potential dropping
                    print(f"  Warning: Flexibility DataFrame became empty after handling duplicates for feature {feature}.")
                    continue

                flex_df.iloc[:, 0] = flex_df.iloc[:, 0].astype(str) # Ensure ID column is string
                # Now, setting index and converting to dict should not warn about unique columns
                flex_df_dict = flex_df.set_index(id_col_name).T.to_dict('list')

            except Exception as e:
                print(f"  Error calculating or processing feature {feature}: {e}")
                traceback.print_exc() # Print traceback for debugging flex issues
                continue # Skip this feature on error

            # Add flexibility features to combined_data
            feature_length_flex = 0
            if flex_df_dict:
                 # Get expected length of flex features + interaction terms
                 first_flex_key = next(iter(flex_df_dict))
                 first_flex_vals = flex_df_dict[first_flex_key]
                 feature_length_flex = len(first_flex_vals) # Scaled
                 if len(first_flex_vals) > 1:
                     feature_length_flex += (len(first_flex_vals) - 1) # Interaction terms

            for seq_id, data in combined_data.items():
                # Ensure 'feat' exists from OHE
                if 'feat' not in data:
                    print(f"  Warning: OHE feature missing for {seq_id}. Cannot add flex features.")
                    continue

                flex_values = flex_df_dict.get(str(seq_id)) # Ensure seq_id is string for lookup
                if flex_values:
                    flex_feat = np.array(flex_values, dtype=np.float32)

                    # Simple Scaling (MinMax to 0-1 range)
                    min_val, max_val = np.min(flex_feat), np.max(flex_feat)
                    scaled_flex_feat = np.zeros_like(flex_feat) # Default to zeros
                    if max_val > min_val:
                        scaled_flex_feat = (flex_feat - min_val) / (max_val - min_val)

                    # Interaction Terms
                    scaled_interaction_terms = np.array([], dtype=np.float32)
                    if len(scaled_flex_feat) > 1:
                        interaction_terms = scaled_flex_feat[:-1] * scaled_flex_feat[1:]
                        min_int, max_int = np.min(interaction_terms), np.max(interaction_terms)
                        if max_int > min_int:
                            scaled_interaction_terms = (interaction_terms - min_int) / (max_int - min_int)
                        else:
                            scaled_interaction_terms = np.zeros_like(interaction_terms) # Handle constant interaction

                    # Concatenate features
                    data['feat'] = np.concatenate([data['feat'],
                                                   scaled_flex_feat,
                                                   scaled_interaction_terms])
                else:
                    # If flex data is missing for this ID, pad with zeros to maintain consistent length
                    # print(f"  Debug: Padding missing flex data for {seq_id}, feature {feature}")
                    if feature_length_flex > 0:
                         padding = np.zeros(feature_length_flex, dtype=np.float32)
                         data['feat'] = np.concatenate([data['feat'], padding])
                    # else: # No flex data was generated at all for this feature, length is 0, no padding needed


    # Prepare feature matrix (X) and target vector (y)
    # Ensure all feature vectors have the same length before creating the matrix
    if not combined_data:
        print("No combined data available before final processing.")
        return np.array([]), np.array([])

    # Determine expected feature length from the first valid entry
    expected_feature_length = None
    valid_data = {}
    processed_ids = set() # Keep track of IDs added to valid_data

    # Find expected length from the first item that has features
    for seq_id, data in combined_data.items():
        if 'feat' in data:
            expected_feature_length = len(data['feat'])
            # print(f"Debug: Determined expected feature length: {expected_feature_length} from seq_id {seq_id}")
            break

    if expected_feature_length is None:
        print("Error: Could not determine expected feature length. No sequences have features.")
        return np.array([]), np.array([])

    # Filter based on the determined length and ensure uniqueness
    for seq_id, data in combined_data.items():
         current_len = len(data.get('feat', []))
         if current_len == expected_feature_length:
             if seq_id not in processed_ids: # Ensure we don't add duplicates from combined_data itself
                 valid_data[seq_id] = data
                 processed_ids.add(seq_id)
         # else:
             # print(f"Debug: Skipping seq_id {seq_id}. Feature length {current_len} != {expected_feature_length}")


    num_removed = len(combined_data) - len(valid_data)
    if num_removed > 0:
        print(f"Warning: {num_removed} sequences removed due to inconsistent feature lengths or missing features.")

    if not valid_data:
        print("No valid data remaining after feature extraction and length check.")
        return np.array([]), np.array([])

    # Create X and y from the filtered valid_data
    # Ensure consistent order by sorting keys (optional but good practice)
    sorted_ids = sorted(valid_data.keys())
    try:
        X = np.array([valid_data[seq_id]['feat'] for seq_id in sorted_ids], dtype=np.float32)
        y = np.array([valid_data[seq_id]['label'] for seq_id in sorted_ids], dtype=np.int32)
    except KeyError as e:
         print(f"Error creating final arrays: Missing key {e}. Check data consistency.")
         return np.array([]), np.array([])
    except ValueError as e:
         print(f"Error creating final arrays: {e}. Likely due to inconsistent sub-array shapes.")
         # Add more debug prints here if needed, e.g., print lengths of individual 'feat' arrays
         # for seq_id in sorted_ids:
         #     print(f"ID: {seq_id}, Length: {len(valid_data[seq_id]['feat'])}")
         return np.array([]), np.array([])


    print(f"Final dataset shape for {tf} - {cell_line}: X={X.shape}, y={y.shape}")

    # Basic sanity check
    if X.shape[0] == 0 or X.shape[0] != y.shape[0]:
        print(f"CRITICAL WARNING: Mismatch or empty arrays. X samples ({X.shape[0]}) vs y labels ({y.shape[0]}) for {tf}-{cell_line}")
        return np.array([]), np.array([])

    return X, y

In [5]:
def train_and_evaluate_with_nested_cv(tf, cell_line, features=('1mer',)):
    """Trains and evaluates XGBoost using nested CV with GPU, avoiding device mismatch issues during prediction."""
    X, y = extract_and_combine_features(tf, cell_line, features)

    if X.shape[0] < 10 or y.shape[0] < 10 or X.ndim != 2 or X.shape[0] != y.shape[0]:
        print(f"Skipping {tf} in {cell_line} with features {features} due to insufficient/invalid data (X:{X.shape}, y:{y.shape})")
        return []  # Return empty list, indicating no results for this combo

    # --- XGBoost Classifier Initialization ---
    pipeline = Pipeline([
        ('classifier', XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            device='cuda',
            random_state=42
        ))
    ])

    param_grid = {
        'classifier__n_estimators': [50, 100, 200, 300],
        'classifier__max_depth': [3, 5, 7, 10],
        'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3],
    }

    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    outer_cv = StratifiedKFold(n_splits=TARGET_FOLDS, shuffle=True, random_state=42)

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=inner_cv,
        scoring='roc_auc',
        n_jobs=1,
        verbose=0,
        error_score='raise'
    )

    fold_results = []
    features_str = "_".join(features)

    print(f"\nStarting Nested CV for {tf} - {cell_line} with features {features_str}")
    outer_cv_iterable = tqdm(outer_cv.split(X, y),
                             total=outer_cv.get_n_splits(),
                             desc=f"Outer CV {tf}-{cell_line} ({features_str})")

    for fold, (train_idx, test_idx) in enumerate(outer_cv_iterable, 1):
        try:
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
                print(f"  Fold {fold}: Skipping fold due to lack of class diversity in train/test split.")
                auc_roc, mcc = np.nan, np.nan
                best_params_str = "Skipped - Class diversity issue"
            else:
                grid_search.fit(X_train, y_train)

                # --- Get trained classifier only (not pipeline) ---
                classifier = grid_search.best_estimator_.named_steps['classifier']

                # --- Ensure test data is on GPU ---
                try:
                    if cp:
                        X_test_gpu = cp.asarray(X_test)
                        print(f"  Fold {fold}: Prediction on device: {X_test_gpu.device}")
                except Exception as e:
                    print(f"  Fold {fold}: Warning - CuPy conversion failed: {e}. Falling back to CPU.")
                    X_test_gpu = X_test

                # --- Predict ---
                y_pred_proba = classifier.predict_proba(X_test_gpu)[:, 1]
                y_pred = classifier.predict(X_test_gpu)

                # --- Metric Calculation ---
                y_test_np = y_test  # y_test is already on CPU
                y_pred_proba_np = cp.asnumpy(y_pred_proba) if cp and isinstance(y_pred_proba, cp.ndarray) else y_pred_proba
                y_pred_np = cp.asnumpy(y_pred) if cp and isinstance(y_pred, cp.ndarray) else y_pred

                try:
                    auc_roc = roc_auc_score(y_test_np, y_pred_proba_np)
                    mcc = matthews_corrcoef(y_test_np, y_pred_np)
                    best_params_str = str(grid_search.best_params_)
                except ValueError as metric_ve:
                    print(f"  Fold {fold}: Metric calculation error: {metric_ve}")
                    auc_roc, mcc = np.nan, np.nan
                    best_params_str = f"Metrics Error: {metric_ve}"
                except Exception as metric_e:
                    print(f"  Fold {fold}: Unexpected error during metric calculation: {metric_e}")
                    auc_roc, mcc = np.nan, np.nan
                    best_params_str = f"Metrics Error: {metric_e}"

        except Exception as fold_e:
            print(f"\n!!!!!!!! ERROR during Fold {fold} for {tf}-{cell_line} ({features_str}) !!!!!!!!!!")
            traceback.print_exc()
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            auc_roc, mcc = np.nan, np.nan
            best_params_str = f"Fold Error: {fold_e}"

        # --- Record Fold Results ---
        result = {
            'tf': tf, 'cell_line': cell_line, 'fold': fold, 'auc_roc': auc_roc,
            'mcc': mcc, 'features': features_str, 'best_params': best_params_str
        }
        fold_results.append(result)

        # --- Save to JSON Lines File ---
        try:
            serializable_result = result.copy()
            serializable_result['auc_roc'] = None if np.isnan(auc_roc) else float(auc_roc)
            serializable_result['mcc'] = None if np.isnan(mcc) else float(mcc)

            with open(JSON_FULL_PATH, 'a') as f_json:
                json.dump(serializable_result, f_json)
                f_json.write('\n')
        except Exception as json_e:
            print(f"  Fold {fold}: Warning - Failed to write result to JSON file {JSON_OUTPUT_FILENAME}: {json_e}")

    # --- Summary ---
    auc_roc_scores = [r['auc_roc'] for r in fold_results if r['auc_roc'] is not None and not np.isnan(r['auc_roc'])]
    mcc_scores = [r['mcc'] for r in fold_results if r['mcc'] is not None and not np.isnan(r['mcc'])]

    print(f"\nFinished Nested CV for {tf} - {cell_line} with features {features_str}")
    if auc_roc_scores:
        avg_auc = np.mean(auc_roc_scores)
        std_auc = np.std(auc_roc_scores)
        avg_mcc = np.mean(mcc_scores)
        std_mcc = np.std(mcc_scores)
        print(f"  Average AUC-ROC: {avg_auc:.4f} ± {std_auc:.4f} ({len(auc_roc_scores)} valid folds)")
        print(f"  Average MCC    : {avg_mcc:.4f} ± {std_mcc:.4f} ({len(mcc_scores)} valid folds)")
    else:
        print(f"  No valid scores obtained across {len(fold_results)} folds.")

    # --- Clean up GPU memory ---
    if cp and 'X_test_gpu' in locals() and isinstance(X_test_gpu, cp.ndarray):
        del X_test_gpu
        cp.get_default_memory_pool().free_all_blocks()

    return fold_results



# def train_and_evaluate_with_nested_cv(tf, cell_line, features=('1mer',)):
#     """Trains and evaluates XGBoost using nested CV with GPU, avoiding device mismatch issues during prediction."""
#     X, y = extract_and_combine_features(tf, cell_line, features)

#     if X.shape[0] < 10 or y.shape[0] < 10 or X.ndim != 2 or X.shape[0] != y.shape[0]:
#         print(f"Skipping {tf} in {cell_line} with features {features} due to insufficient/invalid data (X:{X.shape}, y:{y.shape})")
#         return []  # Return empty list, indicating no results for this combo

#     # --- XGBoost Classifier Initialization ---
#     pipeline = Pipeline([
#         ('classifier', XGBClassifier(
#             objective='binary:logistic',
#             eval_metric='logloss',
#             device='cuda',
#             random_state=42
#         ))
#     ])

#     param_grid = {
#         'classifier__n_estimators': [50, 100, 200, 300],
#         'classifier__max_depth': [3, 5, 7, 10],
#         'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3],
#     }

#     inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
#     outer_cv = StratifiedKFold(n_splits=TARGET_FOLDS, shuffle=True, random_state=42)

#     grid_search = GridSearchCV(
#         estimator=pipeline,
#         param_grid=param_grid,
#         cv=inner_cv,
#         scoring='roc_auc',
#         n_jobs=1,
#         verbose=0,
#         error_score='raise'
#     )

#     fold_results = []
#     features_str = "_".join(features)

#     print(f"\nStarting Nested CV for {tf} - {cell_line} with features {features_str}")
#     outer_cv_iterable = tqdm(outer_cv.split(X, y),
#                              total=outer_cv.get_n_splits(),
#                              desc=f"Outer CV {tf}-{cell_line} ({features_str})")

#     for fold, (train_idx, test_idx) in enumerate(outer_cv_iterable, 1):
#         try:
#             X_train, X_test = X[train_idx], X[test_idx]
#             y_train, y_test = y[train_idx], y[test_idx]

#             if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
#                 print(f"  Fold {fold}: Skipping fold due to lack of class diversity in train/test split.")
#                 auc_roc, mcc = np.nan, np.nan
#                 best_params_str = "Skipped - Class diversity issue"
#             else:
#                 grid_search.fit(X_train, y_train)

#                 # --- Get trained classifier only (not pipeline) ---
#                 classifier = grid_search.best_estimator_.named_steps['classifier']

#                 # --- Ensure test data is on GPU ---
#                 try:
#                     X_test_gpu = cp.asarray(X_test)
#                     print(f"  Fold {fold}: Prediction on device: {X_test_gpu.device}")
#                 except Exception as e:
#                     print(f"  Fold {fold}: Warning - CuPy conversion failed: {e}. Falling back to CPU.")
#                     X_test_gpu = X_test

#                 # --- Predict ---
#                 y_pred_proba = classifier.predict_proba(X_test_gpu)[:, 1]
#                 y_pred = classifier.predict(X_test_gpu)

#                 # --- Metric Calculation ---
#                 y_test_np = y_test  # y_test is already on CPU
#                 y_pred_proba_np = cp.asnumpy(y_pred_proba) if cp and isinstance(y_pred_proba, cp.ndarray) else y_pred_proba
#                 y_pred_np = cp.asnumpy(y_pred) if cp and isinstance(y_pred, cp.ndarray) else y_pred

#                 try:
#                     auc_roc = roc_auc_score(y_test_np, y_pred_proba_np)
#                     mcc = matthews_corrcoef(y_test_np, y_pred_np)
#                     best_params_str = str(grid_search.best_params_)
#                 except ValueError as metric_ve:
#                     print(f"  Fold {fold}: Metric calculation error: {metric_ve}")
#                     auc_roc, mcc = np.nan, np.nan
#                     best_params_str = f"Metrics Error: {metric_ve}"
#                 except Exception as metric_e:
#                     print(f"  Fold {fold}: Unexpected error during metric calculation: {metric_e}")
#                     auc_roc, mcc = np.nan, np.nan
#                     best_params_str = f"Metrics Error: {metric_e}"

#         except Exception as fold_e:
#             print(f"\n!!!!!!!! ERROR during Fold {fold} for {tf}-{cell_line} ({features_str}) !!!!!!!!!!")
#             traceback.print_exc()
#             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#             auc_roc, mcc = np.nan, np.nan
#             best_params_str = f"Fold Error: {fold_e}"

#         # --- Record Fold Results ---
#         result = {
#             'tf': tf, 'cell_line': cell_line, 'fold': fold, 'auc_roc': auc_roc,
#             'mcc': mcc, 'features': features_str, 'best_params': best_params_str
#         }
#         fold_results.append(result)

#         # --- Save to JSON Lines File ---
#         try:
#             serializable_result = result.copy()
#             serializable_result['auc_roc'] = None if np.isnan(auc_roc) else float(auc_roc)
#             serializable_result['mcc'] = None if np.isnan(mcc) else float(mcc)

#             with open(JSON_FULL_PATH, 'a') as f_json:
#                 json.dump(serializable_result, f_json)
#                 f_json.write('\n')
#         except Exception as json_e:
#             print(f"  Fold {fold}: Warning - Failed to write result to JSON file {JSON_OUTPUT_FILENAME}: {json_e}")

#     # --- Summary ---
#     auc_roc_scores = [r['auc_roc'] for r in fold_results if r['auc_roc'] is not None and not np.isnan(r['auc_roc'])]
#     mcc_scores = [r['mcc'] for r in fold_results if r['mcc'] is not None and not np.isnan(r['mcc'])]

#     print(f"\nFinished Nested CV for {tf} - {cell_line} with features {features_str}")
#     if auc_roc_scores:
#         avg_auc = np.mean(auc_roc_scores)
#         std_auc = np.std(auc_roc_scores)
#         avg_mcc = np.mean(mcc_scores)
#         std_mcc = np.std(mcc_scores)
#         print(f"  Average AUC-ROC: {avg_auc:.4f} ± {std_auc:.4f} ({len(auc_roc_scores)} valid folds)")
#         print(f"  Average MCC    : {avg_mcc:.4f} ± {std_mcc:.4f} ({len(mcc_scores)} valid folds)")
#     else:
#         print(f"  No valid scores obtained across {len(fold_results)} folds.")

#     # --- Clean up GPU memory ---
#     if cp and 'X_test_gpu' in locals() and isinstance(X_test_gpu, cp.ndarray):
#         del X_test_gpu
#         cp.get_default_memory_pool().free_all_blocks()

#     return fold_results

### Main ML pipeline to perform training and evaluation 

In [6]:
# Load summary of already completed folds from JSON log
completed_folds_summary = load_completed_folds(JSON_FULL_PATH)

# Feature sets to evaluate
feature_sets = [ ("1mer", ), ("1mer", "DNaseI", "NPP", "twistDisp", "trx", "stiffness")]

# Store all results generated *in this run*
current_run_results = []

# List of TF and cell line pairs to process
tf_cell_line_pairs = []
metadata_file = "/SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/ENCODE_CLF.meta" # Ensure this path is correct


if os.path.exists(metadata_file):
    print(f"Reading metadata from: {metadata_file}")
    try:
        with open(metadata_file) as meta:
            for line in meta:
                rec = line.strip()
                if not rec or rec.startswith('#'): continue # Skip empty/comment lines
                parts = rec.split('_')
                if len(parts) >= 2:
                    tf = parts[0]
                    cline = parts[1]
                    pair = (tf, cline)
                    if pair not in tf_cell_line_pairs:
                        tf_cell_line_pairs.append(pair)
                else:
                    print(f"Skipping malformed line in metadata: {rec}")
    except Exception as meta_e:
         print(f"Error reading metadata file {metadata_file}: {meta_e}")

else:
    print(f"Error: Metadata file not found at {metadata_file}")
    
print(f"Found {len(tf_cell_line_pairs)} TF-Cell Line pairs to process.")
print(f"Processing {len(feature_sets)} feature sets for each pair.")
print(f"Target folds per combination: {TARGET_FOLDS}")

# Note: JSON file is appended to, not cleared, to allow resuming.
# Main loop -------
if tf_cell_line_pairs:
    total_combinations = len(tf_cell_line_pairs) * len(feature_sets)
    processed_count = 0
    skipped_count = 0

    for tf, cell_line in tf_cell_line_pairs:
        for features in feature_sets:
            processed_count += 1
            features_str = "_".join(features)
            current_key = (tf, cell_line, features_str)

            # --- START: Check if combination is already complete --- <<< MODIFICATION
            completed_count = completed_folds_summary.get(current_key, 0)
            if completed_count >= TARGET_FOLDS:
                print(f"\n--- ({processed_count}/{total_combinations}) SKIPPING {tf} - {cell_line} ({features_str}) ---")
                print(f"    Reason: Found {completed_count} completed folds in log (>= target {TARGET_FOLDS}).")
                skipped_count += 1
                continue # Skip to the next feature set or TF/cell line pair
            else:
                 print(f"\n===== ({processed_count}/{total_combinations}) Processing {tf} - {cell_line} ({features_str}) =====")
                 if completed_count > 0:
                     print(f"      (Found {completed_count} existing folds in log, resuming...)")
            # --- END: Check if combination is already complete ---

            try:
                # Run the nested CV for the current combination
                # JSON writing happens inside this function now
                results = train_and_evaluate_with_nested_cv(tf, cell_line, features)
                current_run_results.extend(results) # Add results from this run

            except Exception as e:
                # Log the error and continue with the next pair/feature set
                print(f"\n!!!!!!!! CRITICAL ERROR processing {tf} in {cell_line} with features {features_str} !!!!!!!!!!")
                print(f"Error type: {type(e).__name__}")
                print(f"Error message: {e}")
                traceback.print_exc() # Print full traceback for debugging
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                # Optionally add a placeholder result indicating failure for the CSV
                # We don't write this critical error to JSON here, only fold-level results/errors
                error_result = {
                    'tf': tf, 'cell_line': cell_line, 'fold': -1, 'auc_roc': np.nan,
                    'mcc': np.nan, 'features': features_str, 'best_params': f"Critical Error: {e}"
                }
                current_run_results.append(error_result)
                # No JSON write here for the critical error itself


# --- Save Final Aggregated Results (Original CSV Logic - Now uses current_run_results) ---
if current_run_results:
    results_df = pd.DataFrame(current_run_results)
    # Ensure float precision in CSV output
    output_filename = "nested_cv_gpu_results_THIS_RUN.csv" # Rename to avoid overwriting full history easily
    print(f"\n--- Saving results from THIS RUN ({len(current_run_results)} rows) to {output_filename} ---")
    print(results_df.head())
    try:
        # Convert NaN to empty string or specific marker for CSV if desired
        results_df.fillna({'auc_roc': 'NaN', 'mcc': 'NaN'}, inplace=True) # Or use float_format in to_csv
        results_df.to_csv(output_filename, index=False, float_format='%.6f')
        print("Aggregated CSV results for this run saved.")
    except Exception as csv_e:
        print(f"Error saving aggregated results to CSV: {csv_e}")
else:
    print("\nNo new results were generated in this run (CSV).")

print(f"\nTotal combinations processed/skipped in this run: {processed_count - skipped_count}/{skipped_count}")
print(f"Iterative fold results (including previous runs) are in {JSON_FULL_PATH}")
print("\nScript finished.")

Reading existing results from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/XGB_ENCODE_RESULTS/nested_cv_fold_results.jsonl...
Found 198 combinations with existing results.
Reading metadata from: /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/METADATA/ENCODE_CLF.meta
Found 487 TF-Cell Line pairs to process.
Processing 2 feature sets for each pair.
Target folds per combination: 10

--- (1/974) SKIPPING ATF2 - GM12878 (1mer) ---
    Reason: Found 13 completed folds in log (>= target 10).

--- (2/974) SKIPPING ATF2 - GM12878 (1mer_DNaseI_NPP_twistDisp_trx_stiffness) ---
    Reason: Found 10 completed folds in log (>= target 10).

--- (3/974) SKIPPING ATF2 - H1 (1mer) ---
    Reason: Found 10 completed folds in log (>= target 10).

--- (4/974) SKIPPING ATF2 - H1 (1mer_DNaseI_NPP_twistDisp_trx_stiffness) ---
    Reason: Found 10 completed folds in log (>= target 10).

--- (5/974) SKIPPING ATF2 - HepG2 (1mer) ---
    Reason: Found 10 completed folds in log (>= target 10).


Outer CV EGR1-GM12878 (1mer_DNaseI_NPP_twistDisp_trx_stiffness):   0%|          | 0/10 [00:00<?, ?it/s]

  Fold 1: Prediction on device: <CUDA Device 0>
  Fold 2: Prediction on device: <CUDA Device 0>
  Fold 3: Prediction on device: <CUDA Device 0>
  Fold 4: Prediction on device: <CUDA Device 0>
  Fold 5: Prediction on device: <CUDA Device 0>
  Fold 6: Prediction on device: <CUDA Device 0>
  Fold 7: Prediction on device: <CUDA Device 0>
  Fold 8: Prediction on device: <CUDA Device 0>
  Fold 9: Prediction on device: <CUDA Device 0>
  Fold 10: Prediction on device: <CUDA Device 0>

Finished Nested CV for EGR1 - GM12878 with features 1mer_DNaseI_NPP_twistDisp_trx_stiffness
  Average AUC-ROC: 0.7303 ± 0.0171 (10 valid folds)
  Average MCC    : 0.3399 ± 0.0181 (10 valid folds)

===== (161/974) Processing EGR1 - H1 (1mer) =====
Processing EGR1 - H1...
Final dataset shape for EGR1 - H1: X=(4410, 160), y=(4410,)

Starting Nested CV for EGR1 - H1 with features 1mer


Outer CV EGR1-H1 (1mer):   0%|          | 0/10 [00:00<?, ?it/s]

  Fold 1: Prediction on device: <CUDA Device 0>
  Fold 2: Prediction on device: <CUDA Device 0>
  Fold 3: Prediction on device: <CUDA Device 0>
  Fold 4: Prediction on device: <CUDA Device 0>
  Fold 5: Prediction on device: <CUDA Device 0>
  Fold 6: Prediction on device: <CUDA Device 0>
  Fold 7: Prediction on device: <CUDA Device 0>
  Fold 8: Prediction on device: <CUDA Device 0>
  Fold 9: Prediction on device: <CUDA Device 0>
  Fold 10: Prediction on device: <CUDA Device 0>

Finished Nested CV for EGR1 - H1 with features 1mer
  Average AUC-ROC: 0.6847 ± 0.0259 (10 valid folds)
  Average MCC    : 0.2776 ± 0.0485 (10 valid folds)

===== (162/974) Processing EGR1 - H1 (1mer_DNaseI_NPP_twistDisp_trx_stiffness) =====
Processing EGR1 - H1...
  Calculating flexibility with DNaseI...
  Calculating flexibility with NPP...
  Calculating flexibility with twistDisp...
  Calculating flexibility with trx...
  Calculating flexibility with stiffness...
Final dataset shape for EGR1 - H1: X=(4410, 541

...

  Fold 1: Prediction on device: <CUDA Device 0>
  Fold 2: Prediction on device: <CUDA Device 0>
  Fold 3: Prediction on device: <CUDA Device 0>
  Fold 4: Prediction on device: <CUDA Device 0>
  Fold 5: Prediction on device: <CUDA Device 0>
  Fold 6: Prediction on device: <CUDA Device 0>
  Fold 7: Prediction on device: <CUDA Device 0>
  Fold 8: Prediction on device: <CUDA Device 0>
  Fold 9: Prediction on device: <CUDA Device 0>
  Fold 10: Prediction on device: <CUDA Device 0>

Finished Nested CV for ZSCAN29 - K562 with features 1mer_DNaseI_NPP_twistDisp_trx_stiffness
  Average AUC-ROC: 0.5884 ± 0.0514 (10 valid folds)
  Average MCC    : 0.1104 ± 0.0920 (10 valid folds)

===== (973/974) Processing ZSCAN31 - HepG2 (1mer) =====
Processing ZSCAN31 - HepG2...
Final dataset shape for ZSCAN31 - HepG2: X=(450, 160), y=(450,)

Starting Nested CV for ZSCAN31 - HepG2 with features 1mer


Outer CV ZSCAN31-HepG2 (1mer):   0%|          | 0/10 [00:00<?, ?it/s]

  Fold 1: Prediction on device: <CUDA Device 0>
  Fold 2: Prediction on device: <CUDA Device 0>
  Fold 3: Prediction on device: <CUDA Device 0>
  Fold 4: Prediction on device: <CUDA Device 0>
  Fold 5: Prediction on device: <CUDA Device 0>
  Fold 6: Prediction on device: <CUDA Device 0>
  Fold 7: Prediction on device: <CUDA Device 0>
  Fold 8: Prediction on device: <CUDA Device 0>
  Fold 9: Prediction on device: <CUDA Device 0>
  Fold 10: Prediction on device: <CUDA Device 0>

Finished Nested CV for ZSCAN31 - HepG2 with features 1mer
  Average AUC-ROC: 0.6822 ± 0.0893 (10 valid folds)
  Average MCC    : 0.2815 ± 0.1253 (10 valid folds)

===== (974/974) Processing ZSCAN31 - HepG2 (1mer_DNaseI_NPP_twistDisp_trx_stiffness) =====
Processing ZSCAN31 - HepG2...
  Calculating flexibility with DNaseI...
  Calculating flexibility with NPP...
  Calculating flexibility with twistDisp...
  Calculating flexibility with trx...
  Calculating flexibility with stiffness...
Final dataset shape for ZSCAN

Outer CV ZSCAN31-HepG2 (1mer_DNaseI_NPP_twistDisp_trx_stiffness):   0%|          | 0/10 [00:00<?, ?it/s]

  Fold 1: Prediction on device: <CUDA Device 0>
  Fold 2: Prediction on device: <CUDA Device 0>
  Fold 3: Prediction on device: <CUDA Device 0>
  Fold 4: Prediction on device: <CUDA Device 0>
  Fold 5: Prediction on device: <CUDA Device 0>
  Fold 6: Prediction on device: <CUDA Device 0>
  Fold 7: Prediction on device: <CUDA Device 0>
  Fold 8: Prediction on device: <CUDA Device 0>
  Fold 9: Prediction on device: <CUDA Device 0>
  Fold 10: Prediction on device: <CUDA Device 0>

Finished Nested CV for ZSCAN31 - HepG2 with features 1mer_DNaseI_NPP_twistDisp_trx_stiffness
  Average AUC-ROC: 0.6502 ± 0.0929 (10 valid folds)
  Average MCC    : 0.2436 ± 0.1890 (10 valid folds)

--- Saving results from THIS RUN (7450 rows) to nested_cv_gpu_results_THIS_RUN.csv ---
     tf cell_line  fold   auc_roc       mcc  \
0  EGR1   GM12878     1  0.728327  0.358436   
1  EGR1   GM12878     2  0.760479  0.370910   
2  EGR1   GM12878     3  0.751468  0.355918   
3  EGR1   GM12878     4  0.734387  0.339883  